In [1]:
from math import fabs
from ortools.linear_solver import pywraplp
import random
solver = pywraplp.Solver.CreateSolver('GLOP')
def solve(a,b,c1, c2):
    x = solver.NumVar(0,50,'x')
    y = solver.NumVar(0,50,'y')
    constr1 = solver.Constraint(0,b[0],'ct')
    constr1.SetCoefficient(x,a[0])
    constr1.SetCoefficient(y,a[1])
    constr2 = solver.Constraint(0,b[1],'ct')
    constr2.SetCoefficient(x,a[2])
    constr2.SetCoefficient(y,a[3])
    obje = solver.Objective()
    obje.SetCoefficient(x, c1)
    obje.SetCoefficient(y, c2)
    obje.SetMaximization()
    solver.Solve()
    return c1*x.solution_value()+c2*y.solution_value()
def setparam(a,b,cf,param_a,param_b, c1, c2):
    dia1 = ""
    dia2 = ""
    a[param_a] -= 1
    if cf > solve(a,b,c1,c2):
        a[param_a] += 1
        dia1 += "[" + str(a[param_a]-8) + ".." + str(a[param_a]) + "]"

    else:
        a[param_a] += 2
        if cf > solve(a,b,c1,c2):
            a[param_a] -= 1
            dia1 += "[" + str(a[param_a]) + ".." + str(a[param_a]+8) + "]"

        else:
            return ""

    b[param_b] -= 1
    if cf > solve(a,b,c1,c2):
        b[param_b] += 1
        dia2 += "[" + str(b[param_b]- 8) + ".." +  str(b[param_b]) + "]"
    else:
        b[param_b] += 2
        if cf > solve(a,b,c1,c2):
            b[param_b] -= 1
            dia2 += "[" + str(b[param_b]) + ".." + str(b[param_b]+8) + "]"
        else:
            return ""

    return "a: " + dia1 + " b: " + dia2



while(True):
    constraints_num = 2

    x1 = solver.NumVar(0,50,'x1')# variable creating
    x2 = solver.NumVar(0,50,'x2')

    b1 = random.randrange(2,32,1)
    ct1 = solver.Constraint(0,b1,'ct')
    a1 = random.randrange(1,7,1);
    a2 = random.randrange(-7,7,1);
    ct1.SetCoefficient(x1,a1)
    ct1.SetCoefficient(x2,a2)
    b2 = random.randrange(2,32,1)
    ct2 = solver.Constraint(0,b2,'ct')
    a3 = random.randrange(1,7,1)
    ct2.SetCoefficient(x1,a3)
    if a2 < 0:
        a4 = random.randrange(1,7,1)
    else:
        a4 = random.randrange(-7,7,1)
    ct2.SetCoefficient(x2,a4)

    c1 = random.randrange(1,7,1)
    c2 = random.randrange(1,7,1)
    obj = solver.Objective()
    obj.SetCoefficient(x1, c1)
    obj.SetCoefficient(x2, c2)
    obj.SetMaximization()
    solver.Solve()
    xx1 = x1.solution_value()
    xx2 = x2.solution_value()

    if xx1 != 0 and xx2 != 0 and fabs(round(xx1)-xx1) < 0.00000001 and fabs(round(xx2)-xx2) < 0.00000001 :
        cf = round(c1*xx1+c2*xx2)
        a = [a1,a2,a3,a4]
        b = [b1,b2]
        param_a = random.randrange(0,4,1)
        if param_a < 2:
            param_b = 1
        else:
            param_b = 0
        res = setparam(a,b,cf,param_a,param_b, c1, c2)
        if res == "":
            continue
        else:
            param = [a[param_a],b[param_b]]
            a[param_a] = "a";
            b[param_b] = "b";
            print(str(c1)+"x1 + "+str(c2)+"x2 -> Max")
            print(str(a[0])+"x1 + "+str(a[1])+"x2 <= "+ str(b[0]))
            print(str(a[2])+"x1 + "+str(a[3])+"x2 <= "+ str(b[1]))
            print('Solution:')
            print('Objective value =', cf)
            print('x1 =', round(xx1))
            print('x2 =', round(xx2))
            print("a: "+str(param[0]) + " b:" + str(param[1]))
            print(res)
            print("\n")



5x1 + 6x2 -> Max
2x1 + 5x2 <= b
4x1 + ax2 <= 10
Solution:
Objective value = 17
x1 = 1
x2 = 2
a: 3 b:12
a: [3..11] b: [4..12]


2x1 + 4x2 -> Max
3x1 + ax2 <= 20
4x1 + 1x2 <= b
Solution:
Objective value = 42
x1 = 3
x2 = 9
a: -1 b:21
a: [-9..-1] b: [13..21]


5x1 + 4x2 -> Max
5x1 + 3x2 <= b
6x1 + ax2 <= 12
Solution:
Objective value = 18
x1 = 2
x2 = 2
a: -6 b:16
a: [-14..-6] b: [8..16]


2x1 + 4x2 -> Max
5x1 + 0x2 <= b
4x1 + ax2 <= 3
Solution:
Objective value = 30
x1 = 3
x2 = 6
a: -2 b:15
a: [-10..-2] b: [7..15]


5x1 + 5x2 -> Max
ax1 + -3x2 <= 2
1x1 + 3x2 <= b
Solution:
Objective value = 55
x1 = 7
x2 = 4
a: 2 b:19
a: [2..10] b: [11..19]


4x1 + 5x2 -> Max
2x1 + 0x2 <= b
6x1 + ax2 <= 2
Solution:
Objective value = 138
x1 = 12
x2 = 18
a: -4 b:24
a: [-12..-4] b: [16..24]


1x1 + 3x2 -> Max
1x1 + 0x2 <= b
ax1 + -5x2 <= 20
Solution:
Objective value = 56
x1 = 14
x2 = 14
a: 5 b:14
a: [-3..5] b: [6..14]


5x1 + 5x2 -> Max
1x1 + 0x2 <= b
ax1 + -3x2 <= 27
Solution:
Objective value = 225
x1 = 27
x2 =

KeyboardInterrupt: 